In [ ]:
#Instalacion de dependencias
!pip install transformers datasets gradio wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=febce465df0754ce2a5c92e7ecb10f842a8ea5dfa797e8b0f6e7137cb405dea2
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
#Importación de librerías necesarias
import wikipedia
import re
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import gradio as gr

In [ ]:
#Recoleccion de datos desde Wikipedia
def recolectar_articulos(tema, num_articulos=5):
  wikipedia.set_lang("es")
  articulos = []
  titulos = wikipedia.search(tema, results=num_articulos)
  for titulo in titulos:
    try:
      contenido = wikipedia.page(titulo).content
      articulos.append(contenido)
    except:
      continue
  return "\n".join(articulos)

In [ ]:
#preprocesado del texto
def limpiar_texto(texto):
  texto = re.sub(r'\s+', ' ', texto) #eliminacion de espacios extra
  texto = re.sub(r'\[[^\]]*\]', '', texto) #elimina las referencias [1], [2], ...
  return texto

#recolección y limpieza del corpus
tema = "reciclaje"
texto_crudo = recolectar_articulos(tema)
texto_limpio = limpiar_texto(texto_crudo)

#crear dataset para entrenamiento
datos = {"text": [texto_limpio]}
dataset = Dataset.from_dict(datos)


In [ ]:
#Cargar el modelo preentrenado pequeño
modelo_id = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(modelo_id)
modelo = AutoModelForCausalLM.from_pretrained(modelo_id)

#tokenizar el dataset
def tokenizar_funcion(ejemplo):
  return tokenizer(ejemplo["text"], truncation=True)

dataset_tokenizado = dataset.map(tokenizar_funcion, batched=True)

In [ ]:
#fine-tuning del modelo
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
argumentos_entrenamiento = TrainingArguments(
    output_dir="./modelo_entrenado",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
)

tokenizer.pad_token = tokenizer.eos_token

entrenador = Trainer(
    model=modelo,
    args=argumentos_entrenamiento,
    train_dataset=dataset_tokenizado,
    data_collator=data_collator,
)

#entrenar el modelo
entrenador.train()

In [ ]:
def responder(prompt):
  entradas = tokenizer(prompt, return_tensors="pt")
  salida = modelo.generate(**entradas, max_new_tokens=50)
  respuesta = tokenizer.decode(salida[0], skip_special_tokens=True)
  return respuesta

#interfaz gradio
interfaz = gr.Interface(fn=responder, inputs="text", outputs="text",
                        title="Chat con Modelo Compacto",
                        description="Escribe algo para conversar con el modelo entrenado.")
interfaz.launch()